In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

코랩 사용하시는 분들은 아래의 코드를 실행해주세요!

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [48]:
import os
os.getcwd()

'C:\\Users\\Shon\\Desktop\\연세대학교\\DSL\\세션\\0804_supervised_learning'

코랩 사용시 본인 구글 드라이브 경로는  
/content/drive/MyDrive/...  
입니다.

In [49]:
path = 'C:/Users/Shon/Desktop/연세대학교/DSL/세션/0804_supervised_learning'    # 과제 데이터셋이 저장된 경로를 더 작성해주세요
os.chdir(path)

In [50]:
os.getcwd()

'C:\\Users\\Shon\\Desktop\\연세대학교\\DSL\\세션\\0804_supervised_learning'

사용할 데이터 셋 :  전력 사용량 예측 모델

아래의 과정에서 sklearn의 Pipeline의 사용은 필수는 아닙니다. 선호와 필요에 따라 사용하실 분들은 사용하셔도 좋습니다.

# 데이터

건물 정보와 기후 정보를 활용한 전력사용량 예측 데이터 중에서 

환경 변수(기온, 풍속, 습도, 강수량, 일조)에 대해 전력 사용량을 선형회귀 모델을 사용해서 예측해보고자 합니다.

In [51]:
var_name = ['Temp', 'Wind', 'Moisture', 'Precipitation', 'Sun']
target = 'Wh'

In [52]:
def parsing(data: pd.DataFrame):
    x = data.iloc[:, 3:8]
    x.columns = var_name
    y = data.iloc[:, 2]
    y.name = target
    
    return x, y

In [53]:
def load_data():
    data = pd.read_csv('data/train.csv', encoding='cp949')
    data = data[: 100]
    x, y = parsing(data)    
    cut = int(round(len(x)*0.88))
    tr_x = x[:cut]
    tr_y = y[:cut]
    te_x = x[cut:]
    te_y = y[cut:]
    return tr_x, tr_y, te_x, te_y

In [54]:
train_x, train_y, test_x, test_y = load_data()

## 데이터 확인 

주의! 최종 예측하는 단계 이전에는 현재 가지고 있는 데이터, 즉 train data만을 이용하여 작업을 진행해야 합니다.

현재 가지고 있는 데이터 train_x를 보고 전력 사용량을 예측하는데에 사용할 변수를 선택해주세요.

기초세션에서 다뤘던 내용을 참고하셔서 최적의 변수 조합을 찾아보도록 합시다.

이 때 선택해야하는 변수는 하나일 수도 있고, 여러개의 변수를 사용하여도 좋습니다.

In [55]:
# 데이터 셋을 둘러보는 코드 작성하시고, 왜 좋은지 한번 둘러보세요!
train_x = train_x.loc[:,['Temp','Moisture','Sun']]
train_x

,Temp,Moisture,Sun
0,17.6,92.0,0.0
1,17.7,91.0,0.0
2,17.5,91.0,0.0
3,17.1,91.0,0.0
4,17.0,92.0,0.0
...,...,...,...
83,23.9,68.0,0.6
84,25.0,63.0,0.8
85,26.0,60.0,1.0
86,26.4,60.0,1.0


정답은 없습니다! 

다만 최종 선택한 변수(들)이 왜 선택받았는지 간단히 알려주세요.<br>
답변 : temp, Sun, Moisture
temp와 sun 변수는 높은 상관관계(0.78)를 가지고 있어 선택, Moisture 변수는 앞의 두 변수와는 음의 상관관계를 가지고는 있지만 습도가 높으면 전력을 많이 사용할 것 같아 주관적으로 일단 선택

## 전처리

위에서 선택한 변수들(feature)를 가지고 한번 전력 사용량을 예측하는 모델을 만들어 보기에 앞서,

**train 데이터의 feature의 전처리**를 진행해보려고 합니다. StandardScaler를 사용하셔도 좋고, MinMaxScaler를 사용하셔도 좋습니다! 

또한 PolynomialFeatures를 이용하거나 직접 feature를 계산하여 고차원의 feature를 만드셔도 좋습니다. 이 또한 정답은 없습니다!

이 때 최종 train_x의 shape은 (n_samples, n_features)가 되도록 해주세요.<br>
ex. 4개의 변수를 가진 1000개의 샘플의 경우 train_x.shape은 (1000,4)가 되면 됩니다.

특히 하나의 변수만을 사용하시는 분들은 1D가 아닌 2D가 되도록 신경써주세요<br>
ex. train_x.shape이 (1000,)이 아닌 (1000,1)입니다!

In [11]:
# 사용할 패키지를 import! 표준화 해보니 숫자가 너무 작아서 MinMaxScaler 사용 
from sklearn.preprocessing import MinMaxScaler

In [12]:
# train_x = 
preprocessd_data = MinMaxScaler().fit_transform(train_x)
preprocessd_data

array([[0.20895522, 0.98148148, 0.        ],
       [0.21641791, 0.96296296, 0.        ],
       [0.20149254, 0.96296296, 0.        ],
       [0.17164179, 0.96296296, 0.        ],
       [0.1641791 , 0.98148148, 0.        ],
       [0.15671642, 1.        , 0.        ],
       [0.14179104, 0.94444444, 0.        ],
       [0.15671642, 0.87037037, 0.1       ],
       [0.2238806 , 0.75925926, 0.3       ],
       [0.3358209 , 0.62962963, 0.7       ],
       [0.42537313, 0.42592593, 1.        ],
       [0.54477612, 0.24074074, 1.        ],
       [0.61940299, 0.18518519, 1.        ],
       [0.61940299, 0.05555556, 1.        ],
       [0.65671642, 0.        , 1.        ],
       [0.67164179, 0.01851852, 1.        ],
       [0.71641791, 0.        , 1.        ],
       [0.65671642, 0.03703704, 1.        ],
       [0.58955224, 0.05555556, 1.        ],
       [0.48507463, 0.09259259, 1.        ],
       [0.37313433, 0.22222222, 0.4       ],
       [0.28358209, 0.2962963 , 0.        ],
       [0.

# 모델링

## 학습 

이제 모델링을 해봅시다.

앞서 다룬 여러 regression 모델들(Linear regression, Ridge Regression, Lasso Regression, ..)을 이용해서 fit을 진행해보도록 합시다.

In [13]:
# 사용할 패키지를 import!
from sklearn.linear_model import LinearRegression
#Linear Regression 먼저

In [14]:
# 선형회귀 모델 생성 후 fitting 시킵니다.
# model = 

linear_model = LinearRegression().fit(train_x, train_y) #linear model 불러오기
linear_pred_y = linear_model.predict(train_x)

linear_beta = linear_model.coef_ #계수
linear_intercept = linear_model.intercept_ #y절편
print(f"Beta :{linear_beta}")
print(f"Intercept : {linear_intercept: .4f}")

Beta :[ 79.75823723   8.21085096 -48.94193627]
Intercept :  5958.7064


In [15]:
linear_model.intercept_

5958.706401035408

## 추정

이제 train_x를 이용하여 전력사용량을 추정해보도록 합시다.

In [16]:
esti_y = linear_pred_y
esti_y

array([8117.84966424, 8117.61463701, 8101.66298956, 8069.75969467,
       8069.9947219 , 8070.22974914, 8029.64554882, 8007.85959882,
       8020.58851934, 8063.17314398, 8053.88108726, 8099.38575727,
       8154.51144164, 8097.03548494, 8112.28205069, 8136.44454909,
       8176.08864048, 8128.7037526 , 8065.13219005, 7969.89235984,
       7937.09612245, 7893.8064161 , 7895.45160673, 7888.88594641,
       7890.2961098 , 7883.02536778, 7867.30874757, 7851.35710012,
       7843.61630363, 7787.78553757, 7803.73718502, 7827.19460172,
       7850.65201842, 7889.59102811, 7872.22921726, 7929.23511949,
       7915.63374437, 7912.81341758, 7999.63364567, 8019.77440504,
       8062.26509929, 8101.1778329 , 8077.95544343, 8078.66052513,
       8079.60063406, 8096.72741767, 8105.64335033, 8122.77013394,
       8115.96944638, 8124.41532457, 8141.07205371, 8141.07205371,
       8116.67452808, 8124.88537903, 8093.21711137, 8126.1544456 ,
       8159.18571023, 8222.12563358, 8302.99135264, 8344.77696

# Loss 및 시각화

앞서 fitting하여 얻은 모델을 이용하여 추정한 값으로 MSE를 측정해봅시다. 변수를 하나만 사용하신 분들은 xy 평면에 한번 같이 시각화도 해보셔도 좋습니다.

In [17]:
def mse(pred, true):
    return np.mean((pred-true)**2)

In [18]:
train_mse = mse(esti_y, train_y)
print(f"Linear {train_mse:.5f}")

Linear 3021.80786


In [19]:
def visualize(model, x, y):
    if x.shape[1] != 1:
        return None
    beta = model.coef_[0]
    intercept = model.intercept_
    
    x_axis = np.linspace(np.min(x), np.max(x), 100)
    fit = intercept * x_axis**0 + beta*x_axis 

    plt.figure(figsize=(8,6))
    plt.plot(x_axis, fit, label='Predict', c='r', linewidth=3)
    plt.scatter(x, y, c='blue')
    plt.legend()
    plt.show()

In [20]:
# visualization 하실 분은 진행해주세요! 변수 2개였기에 시각화 x 
# visualize(linear_model, train_x, train_y)

# 예측

테스트셋에 대해서 한번 **예측**해보도록 합시다.

이미 fitting된 모델 (그리고 scaler를 사용하신다면 scaler)에 대해 **fit을 하지 말고** 단순히 predict만 진행해보도록 합시다. 

scaler는 transform만 하시고 model은 predict을 진행해주세요

In [21]:
test_x=test_x.loc[:,['Temp','Moisture','Sun']]

In [22]:
# transform을 하시는 분들은 실행해주세요
#scaler = StandardScaler()
#scaler = scaler.fit(train_data)
#processed_data = scaler.transform(train_data)
#processed_data
# test_x = 

In [23]:
# y1_hat에 predict값 할당
pred_y = linear_model.predict(test_x)

# 성능 평가

한번 학습한 모델에 대해 test_y를 이용하여 mse loss를 측정해도도록 합시다.

In [24]:
def score(train_mse, test_mse):
    print(f"{test_mse/train_mse: .4f}")

In [25]:
test_mse = mse(pred_y, test_y)
print(f"Linear {test_mse:.5f}")

Linear 90162.43334


In [26]:
score(train_mse,test_mse)

 29.8372


In [27]:
# visualization 하실 분은 진행해주세요!
# visualize(model, test_x, test_y)

# 그 이후

이제 여러분들은 한번 모델을 만들어보고 fitting한 다음 예측까지 해보셨습니다.

이제 개선하기 위해 1.데이터부터 5.성능평가 중 수정, 변경하실 부분을 업데이트 해서  최적의 모델을 찾아보세요!

현재 사용하고 있는 scaler, model부터 전처리과정 기타 여러 조건을 조작해보시고 어떤 효과가 있는지 한번 보시고 알려주세요:D

고생하셨습니다!

In [28]:
#이번엔 두 가지 변수 Temp, Sun

In [29]:
# 데이터 셋을 둘러보는 코드 작성하시고, 왜 좋은지 한번 둘러보세요!
train_x = train_x.loc[:,['Temp','Sun']]
train_x

,Temp,Sun
0,17.6,0.0
1,17.7,0.0
2,17.5,0.0
3,17.1,0.0
4,17.0,0.0
...,...,...
83,23.9,0.6
84,25.0,0.8
85,26.0,1.0
86,26.4,1.0


In [30]:
# 사용할 패키지를 import! 표준화 해보니 숫자가 너무 작아서 MinMaxScaler 사용 
from sklearn.preprocessing import MinMaxScaler

In [31]:
# train_x = 
preprocessd_data = MinMaxScaler().fit_transform(train_x)
preprocessd_data

array([[0.20895522, 0.        ],
       [0.21641791, 0.        ],
       [0.20149254, 0.        ],
       [0.17164179, 0.        ],
       [0.1641791 , 0.        ],
       [0.15671642, 0.        ],
       [0.14179104, 0.        ],
       [0.15671642, 0.1       ],
       [0.2238806 , 0.3       ],
       [0.3358209 , 0.7       ],
       [0.42537313, 1.        ],
       [0.54477612, 1.        ],
       [0.61940299, 1.        ],
       [0.61940299, 1.        ],
       [0.65671642, 1.        ],
       [0.67164179, 1.        ],
       [0.71641791, 1.        ],
       [0.65671642, 1.        ],
       [0.58955224, 1.        ],
       [0.48507463, 1.        ],
       [0.37313433, 0.4       ],
       [0.28358209, 0.        ],
       [0.23134328, 0.        ],
       [0.17910448, 0.        ],
       [0.13432836, 0.        ],
       [0.10447761, 0.        ],
       [0.08208955, 0.        ],
       [0.06716418, 0.        ],
       [0.05223881, 0.        ],
       [0.        , 0.        ],
       [0.

In [32]:
# 사용할 패키지를 import!
from sklearn.linear_model import LinearRegression
#Linear Regression 먼저

In [33]:
# 선형회귀 모델 생성 후 fitting 시킵니다.
# model = 

linear_model = LinearRegression().fit(train_x, train_y) #linear model 불러오기
linear_pred_y = linear_model.predict(train_x)

linear_beta = linear_model.coef_ #계수
linear_intercept = linear_model.intercept_ #y절편
print(f"Beta :{linear_beta}")
print(f"Intercept : {linear_intercept: .4f}")

Beta :[  71.40319787 -217.78538838]
Intercept :  6760.1141


In [34]:
linear_model.intercept_

6760.11410516499

In [35]:
esti_y = linear_pred_y
esti_y

array([8016.8103877 , 8023.95070749, 8009.67006791, 7981.10878877,
       7973.96846898, 7966.82814919, 7952.54750962, 7945.04961035,
       7965.75541076, 7985.74605221, 8006.09427315, 8120.33938974,
       8191.74258761, 8191.74258761, 8227.44418655, 8241.72482612,
       8284.56674484, 8227.44418655, 8163.18130846, 8063.21683144,
       8086.78326767, 8088.21358557, 8038.23134706, 7988.24910855,
       7945.40718983, 7916.84591068, 7895.42495132, 7881.14431175,
       7866.86367217, 7816.88143366, 7831.16207324, 7866.86367217,
       7902.56527111, 7966.82814919, 7995.38942834, 8009.67006791,
       7923.98623047, 8009.67006791, 8194.9608029 , 8252.44094068,
       8287.78496014, 8273.86190004, 8231.01998132, 8209.59902195,
       8181.03774281, 8159.61678344, 8138.19582408, 8116.77486472,
       8073.932946  , 8066.79262621, 8059.65230642, 8059.65230642,
       8052.51198664, 8052.51198664, 8016.8103877 , 8001.4570097 ,
       7970.75025369, 8158.18646554, 8287.42738066, 8344.19235

In [36]:
def mse(pred, true):
    return np.mean((pred-true)**2)

In [37]:
train_mse = mse(esti_y, train_y)
print(f"Linear {train_mse:.5f}")

Linear 11423.54088


In [38]:
def visualize(model, x, y):
    if x.shape[1] != 1:
        return None
    beta = model.coef_[0]
    intercept = model.intercept_
    
    x_axis = np.linspace(np.min(x), np.max(x), 100)
    fit = intercept * x_axis**0 + beta*x_axis 

    plt.figure(figsize=(8,6))
    plt.plot(x_axis, fit, label='Predict', c='r', linewidth=3)
    plt.scatter(x, y, c='blue')
    plt.legend()
    plt.show()

In [39]:
# visualization 하실 분은 진행해주세요!  
#visualize(linear_model, train_x, train_y)

In [40]:
test_x=test_x.loc[:,['Temp','Sun']]

In [41]:
# y1_hat에 predict값 할당
pred_y = linear_model.predict(test_x)

In [42]:
def score(train_mse, test_mse):
    print(f"{test_mse/train_mse: .4f}")

In [43]:
test_mse = mse(pred_y, test_y)
print(f"Linear {test_mse:.5f}")

Linear 141209.94067


In [44]:
score(train_mse,test_mse) # Temp, Sun으로만 했던 더 안 좋은 예측 결과가 나왔다(분석 방법은 linear regression)

 12.3613


In [45]:
#Standard Scaler

In [56]:
from sklearn.preprocessing import StandardScaler

In [57]:
scaler = StandardScaler()
scaler = scaler.fit(train_x)
processed_data = scaler.transform(train_x)
processed_data

array([[-8.09422591e-01,  1.32132616e+00, -7.04318071e-01],
       [-7.78799552e-01,  1.25833770e+00, -7.04318071e-01],
       [-8.40045630e-01,  1.25833770e+00, -7.04318071e-01],
       [-9.62537785e-01,  1.25833770e+00, -7.04318071e-01],
       [-9.93160823e-01,  1.32132616e+00, -7.04318071e-01],
       [-1.02378386e+00,  1.38431463e+00, -7.04318071e-01],
       [-1.08502994e+00,  1.19534924e+00, -7.04318071e-01],
       [-1.02378386e+00,  9.43395387e-01, -4.62208734e-01],
       [-7.48176514e-01,  5.65464610e-01,  2.20099397e-02],
       [-2.88830933e-01,  1.24545370e-01,  9.90447288e-01],
       [ 7.86455312e-02, -5.68327722e-01,  1.71677530e+00],
       [ 5.68614150e-01, -1.19821235e+00,  1.71677530e+00],
       [ 8.74844537e-01, -1.38717774e+00,  1.71677530e+00],
       [ 8.74844537e-01, -1.82809698e+00,  1.71677530e+00],
       [ 1.02795973e+00, -2.01706237e+00,  1.71677530e+00],
       [ 1.08920581e+00, -1.95407390e+00,  1.71677530e+00],
       [ 1.27294404e+00, -2.01706237e+00

In [58]:
from sklearn.linear_model import LinearRegression

In [64]:
linear_model = LinearRegression().fit(train_x, train_y) #linear model 불러오기
linear_pred_y = linear_model.predict(train_x)

linear_beta = linear_model.coef_ #계수
linear_intercept = linear_model.intercept_ #y절편
print(f"Beta :{linear_beta}")
print(f"Intercept : {linear_intercept: .4f}")

Beta :[ 79.75823723   8.21085096 -48.94193627]
Intercept :  5958.7064


In [63]:
linear_beta

array([ 79.75823723,   8.21085096, -48.94193627])

In [65]:
def mse(pred, true):
    return np.mean((pred-true)**2)

In [66]:
train_mse = mse(esti_y, train_y)
print(f"Linear {train_mse:.5f}")

Linear 11423.54088


In [67]:
test_x=test_x.loc[:,['Temp','Moisture','Sun']]

In [68]:
# y1_hat에 predict값 할당
pred_y = linear_model.predict(test_x)

In [69]:
def score(train_mse, test_mse):
    print(f"{test_mse/train_mse: .4f}")

In [70]:
test_mse = mse(pred_y, test_y)
print(f"Linear {test_mse:.5f}")

Linear 90162.43334


In [71]:
score(train_mse,test_mse) #standard scaler는 예측값이 좋지 않음

 7.8927
